In [163]:
import tensorflow as tf
import numpy as np
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
a = os.path.join(module_path, "strands_qsr_lib\qsr_lib\src3")

sys.path.append(a)

import progress_learner
import project
from importlib import reload
import config

np.random.seed(1234)

In [164]:
project_name = "SlideAround"

p_name = project_name.lower() + "_project.proj"

p = project.Project.load(os.path.join('..' , 'learned_models', p_name))

In [47]:
inputs = [[ 0.,-1.,-1.,0.,0.,0.,-0.5],
 [ 0.,-1.,  -1.,   0.,   0.,   0.,  -0.5],
 [ 0.,0.,  -1.,   0.,   0.,   0.,  -0.5],
 [ 0.,-1.,  -1.,   0.,   0.,   0.,  -0.5],
 [ 0.,0.,  -1.,   0.,   0.,   0.,  -0.5],
 [ 0.,-1.,  -1.,   0.,   0.,   0.,   1. ],
 [ 0.,-1.,  -1.,   0.,   0.,   0.,   1. ],
 [ 0.,0.,  -1.,   0.,   0.,   0.,   1. ],
 [ 0.,-1.,  -1.,   0.,   0.,   0.,   1. ],
 [ 0.,0.,  -1.,   0.,   0.,   0.,   1. ],
 [ 0.,-1.,  -1.,   0.,   0.,   0.,   1. ],
 [ 0.,-1.,  -1.,   0.,   0.,   0.,   1. ],
 [ 0.,0.,  -1.,   0.,   0.,   0.,   0.5],
 [ 0.,-1.,  -1.,   0.,   0.,   0.,   0.5],
 [ 0.,0.,  -1.,   0.,   0.,   0.,   0.5],
 [ 0.,-1.,  -1.,   0.,   0.,   0.,   0.5],
 [ 0.,0.,  -1.,   0.,   0.,   0.,   0.5],
 [ 0.,-1.,  -1.,   0.,   0.,   0.,   0.5],
 [-1.,-1.,  -1.,   0.,   0.,   0.,   0. ],
 [ 0.,0.,  -1.,   0.,   0.,   0.,   0. ]]

In [210]:
reload(config)
reload(progress_learner)
tf.reset_default_graph()

c = config.Config()
c.num_episodes = 10
c.keep_prob_bool = False

global_step = tf.Variable(0, name="global_step", trainable=False)

sess =  tf.Session()

# sess.run(tf.global_variables_initializer())

In [211]:
with tf.variable_scope("model") as scope:
    print('-------- Load progress model ---------')
    pe = progress_learner.EventProgressEstimator(is_training=False,
                                                is_dropout = False,
                                                 name = project_name, 
                                                 config = c)  

# Print out all variables that would be restored
for variable in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=''):
    print (variable.name)

all_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='model/' + project_name)
state_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='model/' + project_name + '/state')

all_but_state_variables = [v for v in all_variables if not v in state_variables]

saver = tf.train.Saver(all_but_state_variables)

saver.restore(sess, os.path.join('..' , 'learned_models', 'progress_' + project_name + '.mod.1'))

-------- Load progress model ---------
self.inputs.shape = (?, 20, 200)  after linear layer
output.shape = (30, 200) after LSTM
self.output.shape = (30,) after linear
self._targets.shape = (?,) 
global_step:0
model/SlideAround/state/Variable:0
model/SlideAround/state/Variable_1:0
model/SlideAround/state/Variable_2:0
model/SlideAround/state/Variable_3:0
model/SlideAround/linear/weight:0
model/SlideAround/linear/bias:0
model/SlideAround/lstm/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0
model/SlideAround/lstm/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0
model/SlideAround/output_linear/weight:0
model/SlideAround/output_linear/bias:0
INFO:tensorflow:Restoring parameters from ..\learned_models\progress_SlideAround.mod.1


In [218]:
pe.reset_state( sess = sess )

LSTMStateTuple(c=<tf.Variable 'model/SlideAround/state/Variable:0' shape=(30, 200) dtype=float32_ref>, h=<tf.Variable 'model/SlideAround/state/Variable_1:0' shape=(30, 200) dtype=float32_ref>)
LSTMStateTuple(c=<tf.Variable 'model/SlideAround/state/Variable_2:0' shape=(30, 200) dtype=float32_ref>, h=<tf.Variable 'model/SlideAround/state/Variable_3:0' shape=(30, 200) dtype=float32_ref>)


In [219]:
o_inputs = np.repeat(np.expand_dims(inputs, axis = 0), 30, axis = 0)

current_progress = pe.predict(o_inputs, sess = sess)
print ('current_progress', current_progress[0])

current_progress 0.005496383
